In [ ]:
from ai_api_client_sdk.ai_api_v2_client import AIAPIV2Client
import requests
import json
import pprint
#key file from service instance is necessary to login
with open("ai-core-key.txt") as file:
    aicorekey = json.load(file)

In [ ]:
AUTH_URL = aicorekey["url"]
AI_API = aicorekey["serviceurls"]["AI_API_URL"] + "/v2"
CLIENT_ID = aicorekey["clientid"]
CLIENT_SECRET = aicorekey["clientsecret"]
RESOURCE_GROUP = "llm"
SCENARIO_ID = "transformers"
SERVING_EXECUTABLE = "transformers"

In [ ]:
ai_api_v2_client = AIAPIV2Client(
    base_url=AI_API + "/lm", 
    auth_url=AUTH_URL + "/oauth/token", 
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET, 
    resource_group=RESOURCE_GROUP
)

Create Serving Configuration

In [ ]:
serving_configuration = {
    "name": "transformers config",
    "scenario_id": "transformers",
    "executable_id": "transformers",
    "parameter_bindings": [ ],
    "input_artifact_bindings": [ ]
}

serving_config_resp = ai_api_v2_client.configuration.create(**serving_configuration)
assert serving_config_resp.message == 'Configuration created'

Create Deployment

In [ ]:
deployment_resp = ai_api_v2_client.deployment.create(serving_config_resp.id)
deployment_resp.id

In [ ]:
deployment = ai_api_v2_client.deployment.get(deployment_resp.id)
deployment.status_details   # check status

Inference

In [ ]:
def build_llama2_prompt(role_prompt, task_prompt):
    B_S, E_S = "<s>", " </s>"
    B_INST, E_INST = "[INST]", " [/INST]\n"
    B_SYS, E_SYS = " <<SYS>>\n", "\n<</SYS>>\n\n"
    SYSTEM_PROMPT = B_SYS + role_prompt + E_SYS
    return B_S + B_INST + SYSTEM_PROMPT + task_prompt + E_INST

In [ ]:
def get_response(full_prompt, args={}):
    res = requests.post(
        f"https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/{deployment_resp.id}/v2/predict",
        json={"prompt": full_prompt, "args": args},
        headers={
            "Authorization": ai_api_v2_client.rest_client.get_token(),
            "ai-resource-group": RESOURCE_GROUP,
            "Content-Type": "application/json"
        })
    if res.status_code != 200:
        raise Exception("ERROR WITH DEPLOYMENT " + str(res.status_code) + " " + str(res.content))
    return res.json()[0]["generated_text"]   

In [ ]:
r = get_response(build_llama2_prompt(role_prompt="You are a poet!", task_prompt="Write a 5 line Poem, about lamas!"))

In [ ]:
print(r)

In [ ]:
build_llama2_prompt(role_prompt="You are a poet!", task_prompt="Write a 5 line Poem, about lamas!")